
# Topic extraction with Non-negative Matrix Factorization and Latent Dirichlet Allocation

This is an example of applying :class:`~sklearn.decomposition.NMF` and
:class:`~sklearn.decomposition.LatentDirichletAllocation` on a corpus
of documents and extract additive models of the topic structure of the
corpus.  The output is a plot of topics, each represented as bar plot
using top few words based on weights.

Non-negative Matrix Factorization is applied with two different objective
functions: the Frobenius norm, and the generalized Kullback-Leibler divergence.
The latter is equivalent to Probabilistic Latent Semantic Indexing.

The default parameters (n_samples / n_features / n_components) should make
the example runnable in a couple of tens of seconds. You can try to
increase the dimensions of the problem, but be aware that the time
complexity is polynomial in NMF. In LDA, the time complexity is
proportional to (n_samples * iterations).


In [ ]:
from time import time

import matplotlib.pyplot as plt

from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer

import pandas as pd
import numpy as np

In [ ]:
#########################################################################################################
# This code is from the scikit-learn example                                                            #
# https://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html   #
# Author: Olivier Grisel <olivier.grisel@ensta.org>                                                     #
#         Lars Buitinck                                                                                 #
#         Chyi-Kwei Yau <chyikwei.yau@gmail.com>                                                        #
# License: BSD 3 clause                                                                                 #
#########################################################################################################
n_samples = 2000
n_features = 1000
n_components = 10
n_top_words = 20
batch_size = 128
init = "nndsvda"


def plot_top_words(model, feature_names, n_top_words, title):
    fig, axes = plt.subplots(2, 5, figsize=(30, 15), sharex=True)
    axes = axes.flatten()
    for topic_idx, topic in enumerate(model.components_):
        top_features_ind = topic.argsort()[-n_top_words:]
        top_features = feature_names[top_features_ind]
        weights = topic[top_features_ind]

        ax = axes[topic_idx]
        ax.barh(top_features, weights, height=0.7)
        ax.set_title(f"Topic {topic_idx +1}", fontdict={"fontsize": 30})
        ax.tick_params(axis="both", which="major", labelsize=20)
        for i in "top right left".split():
            ax.spines[i].set_visible(False)
        fig.suptitle(title, fontsize=40)

    plt.subplots_adjust(top=0.90, bottom=0.05, wspace=0.90, hspace=0.3)
    plt.show()

In [ ]:
data = pd.read_csv('./uiuc_top_all.csv')
# data_controversial = pd.read_csv('./uiuc_controversial_all.csv')
# data = pd.concat([data, data_controversial])

# replace NaN with empty string or 0 for numerical columns
data['PostText'] = data['PostText'].replace(np.nan, '', regex=True)
data['Title'] = data['Title'].replace(np.nan, '', regex=True)
data['Sentiment'] = data['Sentiment'].replace(np.nan, 0, regex=True)

years = [str(i) for i in range(2008, 2024)]

data_by_year = {year: data[data['Date'].str.contains(year)] for year in years}

# remove years with less than 2 posts
data_by_year = {year: data_by_year[year] for year in years if len(data_by_year[year]) > 10}
years = list(data_by_year.keys())

print(years)
data

In [ ]:
year_model = {}
for year in years:
    df = data_by_year[year]

    # concat post titles to body text (data)
    post_titles = df['Title'].to_list()
    post_body = df['PostText'].to_list()

    dataset = [str(title + ' ' + body) for title, body in zip(post_titles, post_body)]

    data_samples = dataset[:n_samples]
    # print([type(d) for d in data_samples])

    #########################################################################################################
    # This code is from the scikit-learn example                                                            #
    #########################################################################################################
    # Use tf-idf features for NMF.
    print("Extracting tf-idf features for NMF...")
    tfidf_vectorizer = TfidfVectorizer(
        max_df=0.95, min_df=2, max_features=n_features, stop_words="english"
    )
    t0 = time()
    tfidf = tfidf_vectorizer.fit_transform(data_samples)
    print("done in %0.3fs." % (time() - t0))

    # Fit the NMF model
    print(
        "\n" * 2,
        "Fitting the NMF model (generalized Kullback-Leibler "
        "divergence) with tf-idf features, n_samples=%d and n_features=%d..."
        % (n_samples, n_features),
    )
    t0 = time()
    nmf = NMF(
        n_components=n_components,
        random_state=1,
        init=init,
        beta_loss="kullback-leibler",
        solver="mu",
        max_iter=1000,
        alpha_W=0.00005,
        alpha_H=0.00005,
        l1_ratio=0.5,
    ).fit(tfidf)
    print("done in %0.3fs." % (time() - t0))

    tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()
    # plot_top_words(
    #     nmf,
    #     tfidf_feature_names,
    #     n_top_words,
    #     "Topics in NMF model (generalized Kullback-Leibler divergence)",
    # )
    ###############################################################################
    #                           End of scikit-learn example                       #
    ###############################################################################
    year_model[year] = (nmf, tfidf_feature_names)

In [100]:
import json
topics = []
posts_by_topic = {}
for year, (nmf, tfidf_feature_names) in year_model.items():
    # print(f"Year: {year}")
    for topic_idx, topic in enumerate(nmf.components_):
        top_features_ind = topic.argsort()[-3:]
        top_features = tfidf_feature_names[top_features_ind]
        # print(f"Topic {topic_idx + 1}: {', '.join(top_features)}")

        # Get the posts in the year's data that include the top features
        posts_with_top_features = data_by_year[year][data_by_year[year]['PostText'].str.contains('|'.join(top_features))]

        posts_by_topic[", ".join(top_features)] = [
           { "title": title, "url": url } 
           for title, url in zip(posts_with_top_features['Title'], posts_with_top_features['URL'])]

        proportion_posts = len(posts_with_top_features)

        # Calculate the sentiment score for each post and sum them
        sentiment_score = sum(posts_with_top_features['Sentiment'])

        # print(f"Sentiment Score: {sentiment_score}")

        topics.append(
            {
                "year": int(year),
                "words": ", ".join(top_features),
                "sentiment": sentiment_score,
                "count": proportion_posts,
            }
        )

normalized_topics = []
for year in years:
    year_topics = [t for t in topics if t['year'] == int(year)]
    ranking = np.argsort([t['count'] for t in year_topics])
    for i, t in enumerate(year_topics):
        t['rank'] = int(ranking[i])
        normalized_topics.append(t)

with open('topics.json', 'w') as f:
  json.dump(normalized_topics, f)

with open('posts_by_topic.json', 'w') as f:
  json.dump(posts_by_topic, f)